## HCBB gryphons_goal_3

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time

%matplotlib inline

## 1. What procedures could be reasonably associated with each individual Provider Type? (There are 88 types of Providers.)
- Filters for Physician/Other Supplier --Do we need a "credentials of the provider" column?
    - Entity Type = I
    - Place of Service = F and/or O
- What is the normal ‘basket’ of procedures for each Provider Type?

## 2. Which procedure codes drive the most revenue for each Professional Provider Types?
- Exclude office visits

## Stretch: Create a process that predicts if an individual professional should be Provider Type ‘Orthopedic Surgery’ based on the procedure codes they have data for.
- How often does this data-driven expected Provider Type of ‘Orthopedic Surgery’ differ from the listed Provider Type for given professional?

## Import pkl data, from first project notebook

In [2]:
providers_and_procedures = pd.read_pickle("../data/hcpcs_providers_clean.pkl")

## begin EDA of dataset

In [ ]:
providers_and_procedures.info()

In [4]:
providers_and_procedures.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Credentials of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,325.780000,56.827200,57.492400
1,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,449.000000,78.460000,79.306154
2,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,614.000000,102.807692,103.895385
3,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,755.932203,141.293559,142.865763
4,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16,16,101.680000,462.812500,79.710000,80.750000


In [5]:
providers_and_procedures['Place of Service'].value_counts()

O    5740315
F    3675810
Name: Place of Service, dtype: int64

In [ ]:
providers_and_procedures['Provider Type'].unique()

In [ ]:
providers_and_procedures['HCPCS Description'].value_counts()

## Begin work on Question 1. 
- create pivot table for provider types and HCPCS descriptions

In [8]:
providers_pivot_table = pd.pivot_table(providers_and_procedures, index = ['Provider Type', 'HCPCS Code', 
                                                                          'HCPCS Description', 'Place of Service'])

In [ ]:
providers_pivot_table.head(75)

In [10]:
providers_pivot_table.shape

(56209, 8)

## Attempt subset of one Provider Type - 'Addiction Medicine'

In [11]:
addict_med_type = providers_and_procedures.loc[providers_and_procedures['Provider Type'] == 'Addiction Medicine']

In [12]:
addict_med_type.shape

(803, 23)

In [ ]:
addict_med_type.head()

In [14]:
addiction_pivot_table = pd.pivot_table(addict_med_type, index = ['Provider Type', 'HCPCS Description'])

In [15]:
addiction_pivot_table.shape

(139, 8)

## Turn full pivot table into a dataframe

In [ ]:
## df = table.reset_index() 
prov_desc_df = providers_pivot_table.reset_index()
prov_desc_df.head(5)

In [17]:
prov_desc_df['HCPCS Description'].value_counts()

Vaccine for influenza for administration into muscle, 0.5 ml dosage                                          215
Ultrasound study of arteries of both arms and legs                                                           159
Injections of lower or sacral spine facet joint using imaging guidance                                       154
Established patient office or other outpatient, visit typically 25 minutes                                   144
Established patient office or other outpatient visit, typically 15 minutes                                   144
                                                                                                            ... 
Destruction of 1 or more growths on liver, open procedure                                                      1
Insertion of hardware to stabilize broken thigh bone or separated growth plate, accessed through the skin      1
Closed treatment of thigh bone fracture with manipulation                                       

## Delete HCPCS codes for office visits

In [18]:
prov_desc_df_no_office = prov_desc_df.loc[~prov_desc_df['HCPCS Code'].str.contains('992..|993..', regex=True)]

In [19]:
prov_desc_df_no_office.shape

(51680, 12)

In [ ]:
prov_desc_df_no_office

## Calculate percentage of the services provided for each type.

In [21]:
## sort the provider types with the number of services ranked 
prov_desc_df_sorted = prov_desc_df_no_office.sort_values(by = ['Provider Type', 'Number of Services'], ascending = (True, False))

In [ ]:
prov_desc_df_sorted

In [23]:
## makes a list of datframes for each single provider type with its hcpcs codes from the original df
provider_type_df_list = list(prov_desc_df_sorted.groupby(['Provider Type'])) 

In [24]:
len(provider_type_df_list) ## check for 88 provider types--one is lost from dropping office visits

87

In [ ]:
provider_type_df_list[0]

In [26]:
## iterate over list
provider_threshold_list = []

for group, provider in provider_type_df_list:
    total_hcpcs = len(provider) ## number of hcpcs for each provider
    keep_rows = round(total_hcpcs * .80) ## can change this number to keep desired percentage
    keep_provider = provider.head(keep_rows)
    provider_threshold_list.append(keep_provider)

providers_final = pd.concat(provider_threshold_list)

In [27]:
len(provider_threshold_list)

87

In [28]:
## Use to export to Tableau
providers_final_df = providers_final.reset_index()

In [52]:
## Drop extra index column...
providers_final_df.drop(columns = ['index']) ##this did not remove extra 'index' column
providers_final_df.head()

,index,Provider Type,HCPCS Code,HCPCS Description,Place of Service,Average Medicare Allowed Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,Average Submitted Charge Amount,National Provider Identifier,Number of Distinct Medicare Beneficiary/Per Day Services,Number of Medicare Beneficiaries,Number of Services
0,71,Addiction Medicine,90847,"Family psychotherapy including patient, 50 min...",O,102.760000,75.680820,80.779673,120.013687,1.700878e+09,1622.0,128.0,1622.0
1,83,Addiction Medicine,95004,"Injection of allergenic extracts into skin, ac...",O,6.360746,4.800754,5.395866,8.425672,1.295891e+09,39.0,39.0,1340.0
2,33,Addiction Medicine,82575,Creatinine clearance measurement to test for k...,O,12.970000,12.710000,12.710000,25.000000,1.942314e+09,957.0,41.0,957.0
3,156,Addiction Medicine,G0397,Alcohol and/or substance (other than tobacco) ...,O,69.620000,53.877956,55.292240,139.240000,1.124073e+09,817.0,283.0,817.0
4,80,Addiction Medicine,93798,Physician services for outpatient heart rehabi...,F,13.580000,10.207731,10.823529,100.000000,1.740270e+09,595.0,31.0,595.0


In [30]:
## Create dataframe for Office ('O') "Place of Service"

In [31]:
prov_desc_df_office = prov_desc_df.loc[prov_desc_df['Place of Service'] == 'O']

In [32]:
##prov_desc_df_office = prov_desc_df.sort_values(by = 'Number of Services', ascending = False)

In [ ]:
prov_desc_df_office.head()

## Check there are no "F" (facility) 'Place of Service' entries.

In [34]:
prov_desc_df_office['Place of Service'].value_counts()

O    35001
Name: Place of Service, dtype: int64

## Use a dictionary for-loop to sort HCPCS descriptions by Provider Type for Office Visits.

In [ ]:
# Initialize an empty dictionary for office visits: 

provider_type_dict_ofc = {} 

# Iterate over the file
    
    # Iterate over the column in DataFrame
for key, val in prov_desc_df_office.iterrows():
    if val['Provider Type'] not in provider_type_dict_ofc.keys():
        provider_type_dict_ofc[val['Provider Type']]=[val['HCPCS Description']]
        
    elif (val['HCPCS Description'] not in provider_type_dict_ofc[val['Provider Type']]):
        provider_type_dict_ofc[val['Provider Type']].append(val['HCPCS Description'])
        
    else:
        continue


# Print the populated dictionary
print(provider_type_dict_ofc)

## Turn provider type/descriptions dictionary into a dataframe

In [36]:
provider_type_subset_ofc = pd.DataFrame.from_dict(provider_type_dict_ofc, orient = "index")

In [ ]:
pd.options.display.max_rows = 100
provider_type_subset_ofc

## Repeat the above steps to subset the "F" ('Facility') Services 

In [38]:
prov_desc_df_facility = prov_desc_df.loc[prov_desc_df['Place of Service'] == 'F']

In [39]:
##prov_desc_df_facility = prov_desc_df.sort_values(by = 'Number of Services', ascending = False)

In [40]:
prov_desc_df_facility.head()

,Provider Type,HCPCS Code,HCPCS Description,Place of Service,Average Medicare Allowed Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount,Average Submitted Charge Amount,National Provider Identifier,Number of Distinct Medicare Beneficiary/Per Day Services,Number of Medicare Beneficiaries,Number of Services
1,Addiction Medicine,36556,Insertion of central venous catheter for infus...,F,112.510000,88.210000,97.910000,129.39,1.770876e+09,11.0,11.0,12.0
2,Addiction Medicine,36620,Insertion of arterial catheter for blood sampl...,F,47.990000,37.620000,41.360000,55.19,1.770876e+09,13.0,12.0,13.0
3,Addiction Medicine,70100,"X-ray of mandible, less than 4 views",F,9.078696,6.833478,7.167391,35.00,1.306066e+09,46.0,39.0,46.0
5,Addiction Medicine,70310,"X-ray of teeth, less than full mouth",F,8.043478,6.039783,6.335000,20.00,1.306066e+09,46.0,39.0,46.0
7,Addiction Medicine,70371,Imaging of voice box with speech evaluation,F,41.515217,31.208478,31.936739,70.00,1.306066e+09,46.0,39.0,46.0


In [41]:
## confirm only 'Facility' entries

prov_desc_df_facility['Place of Service'].value_counts()

F    21208
Name: Place of Service, dtype: int64

## Use a dictionary for-loop to sort HCPCS descriptions by Provider Type for Facilty Visits.

In [ ]:
# Initialize an empty dictionary for facility visits: 

provider_type_dict_fac = {} 

# Iterate over the file
    
    # Iterate over the column in DataFrame
for key, val in prov_desc_df_facility.iterrows():
    if val['Provider Type'] not in provider_type_dict_fac.keys():
        provider_type_dict_fac[val['Provider Type']]=[val['HCPCS Description']]
        
    elif (val['HCPCS Description'] not in provider_type_dict_fac[val['Provider Type']]):
        provider_type_dict_fac[val['Provider Type']].append(val['HCPCS Description'])
        
    else:
        continue


# Print the populated dictionary
print(provider_type_dict_fac)

In [43]:
provider_type_subset_fac = pd.DataFrame.from_dict(provider_type_dict_fac, orient = "index")

In [ ]:
pd.options.display.max_rows = 100
provider_type_subset_fac

## There is a discrepancy between number of codes and the descriptions...

In [45]:
prov_desc_df['HCPCS Description'].nunique()

5171

In [46]:
prov_desc_df['HCPCS Code'].nunique()

5702

## Created csv files for export to Tableau to further analysis

In [47]:
## Main dataframe
prov_desc_df_no_office.to_csv('prov_type_no_office_visit.csv')

In [48]:
## Office services subset
provider_type_subset_ofc.to_csv('prov_type_sub_ofc.csv')

In [49]:
## Facility services subset
provider_type_subset_fac.to_csv('prov_type_sub_fac.csv')

In [50]:
## Office visits dataframe
prov_desc_df_office.to_csv('provider_type_office_service.csv')

## Attempt to use plot(s) to visualize some parts of the data

In [ ]:
"""fig, ax = plt.subplots(figsize=(8,8))
plt.bar(prov_desc_df['Provider Type'], prov_desc_df['Place of Service'])
plt.xlabel('Type')
plt.ylabel('Place of Service')
##plt.legend()
##  .set_xticklabels(years)
plt.title('Provider Types by Location', color = 'black');"""